In [1]:
def primes(kmax):  
    p = []
    result = []  
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [3]:
t_py = %timeit -o p = primes(100)

1000 loops, best of 3: 792 µs per loop


In [6]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


Start with simplest cython

In [7]:
%%cython -a
def primes_simplecython(kmax):  
    p = []
    result = []  
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [8]:
t_cy0 = %timeit -o p = primes_simplecython(100)

1000 loops, best of 3: 410 µs per loop


Now for proper cythonization. Add annotation (`-a`) if you want!

In [12]:
%%cython -a
def primes_cython(int kmax):  # The argument will be converted to int or raise a TypeError.
    cdef int n, k, i  # These variables are declared with C types.
    cdef int p[1000]  # Another C type
    result = []  # A Python type
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p[k] = n
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [13]:
t_cy = %timeit -o p = primes_cython(100)

10000 loops, best of 3: 35.2 µs per loop


Factor 20 in speedup

Finally, let's compare with just-in-time compilation with numba

In [19]:
from numba import jit, vectorize, float64

In [20]:
@jit
def primes_jit(kmax):  
    p = []
    result = []  
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [21]:
t_jit = %timeit -o p = primes_jit(100)

The slowest run took 4420.77 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 92 µs per loop


In [22]:
print(" Python: %.3E\n Simply Cython: %.3E\n Proper Cython: %.3E\n Numba-jit: %.3E"%(t_py.best,t_cy0.best,t_cy.best,t_jit.best))

 Python: 7.920E-04
 Simply Cython: 4.097E-04
 Proper Cython: 3.521E-05
 Numba-jit: 9.203E-05


Just-in-time compilation comes close to Cython

Notice that the slowest run took much much longer than the fastest.  
`cache=True` stores the compiled function in file-based cache and avoids re-compilation on re-running


In [23]:
@jit(cache=True)
def primes_jit2(kmax):  
    p = []
    result = []  
    if kmax > 1000:
        kmax = 1000
    k = 0
    n = 2
    while k < kmax:
        i = 0
        while i < k and n % p[i] != 0:
            i = i + 1
        if i == k:
            p.append(n)
            k = k + 1
            result.append(n)
        n = n + 1
    return result

In [24]:
%%timeit
p = primes_jit2(100)

The slowest run took 115.25 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 65.3 µs per loop
